In [1]:
# -*- coding: utf-8 -*-
"""
   Introduction to Deep Learning
   Assignment 3: Sentiment Classification of Tweets on a Recurrent Neural Network using Pretrained Embeddings

   Hande Celikkanat

   Credit: Data preparation pipeline adopted from https://medium.com/@sonicboom8/sentiment-analysis-torchtext-55fb57b1fab8
"""

'\n   Introduction to Deep Learning\n   Assignment 3: Sentiment Classification of Tweets on a Recurrent Neural Network using Pretrained Embeddings\n\n   Hande Celikkanat\n\n   Credit: Data preparation pipeline adopted from https://medium.com/@sonicboom8/sentiment-analysis-torchtext-55fb57b1fab8\n'

In [1]:
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import spacy
import regex as re
from torchtext import vocab
import time
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
# Constants - Add here as you wish
N_EPOCHS = 5
EMBEDDING_DIM = 200 
OUTPUT_DIM = 2

In [4]:
# Auxilary functions for data preparation
tok = spacy.load('en_core_web_sm',disable=['parser', 'tagger', 'ner'])
def tokenizer(s):
    return [w.text.lower() for w in tok(tweet_clean(s))]

def tweet_clean(text):
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text) # remove non alphanumeric character
    text = re.sub(r'https?:/\/\S+', ' ', text) # remove links
    return text.strip()

In [5]:
# Evaluation functions
def get_accuracy(output, gold):
    _, predicted = torch.max(output, dim=1)
    correct = torch.sum(torch.eq(predicted,gold)).item()
    acc = correct / gold.shape[0]
    return acc


def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.TweetText
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.Label)
            acc = get_accuracy(predictions, batch.Label)
            epoch_loss += loss.item()
            epoch_acc += acc

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [6]:
# Utility
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [154]:
# Recurrent Network
class RNN(nn.Module):
    def __init__(self, hidden_dim, batch_size):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(30116, EMBEDDING_DIM) # 30116 tulee pretrained_embeddingsien koosta (vocab_size)
        self.lstm = nn.LSTM(EMBEDDING_DIM, hidden_dim, num_layers=5)
        self.fc = nn.Linear(hidden_dim, (OUTPUT_DIM))

    def forward(self, x, seq, hidden=None):
        embeds = self.embedding(x)
        #print("embeds shape", embeds.shape)
        out, _ = self.lstm(embeds)
        #print("LSTM shape", out.shape)
        x = self.fc(out[-1])
        #print("Linear shape ", x.shape)
        x = F.log_softmax(x, dim =1)
        return x

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# --- Data Preparation ---

# define the columns that we want to process and how to process
txt_field = torchtext.data.Field(sequential=True,
                                 tokenize=tokenizer,
                                 include_lengths=True,
                                 use_vocab=True)
label_field = torchtext.data.Field(sequential=False,
                                   use_vocab=False)

csv_fields = [
    ('Label', label_field), # process this field as the class label
    ('TweetID', None), # we dont need this field
    ('Timestamp', None), # we dont need this field
    ('Flag', None), # we dont need this field
    ('UseerID', None), # we dont need this field
    ('TweetText', txt_field) # process it as text field
]

train_data, dev_data, test_data = torchtext.data.TabularDataset.splits(path='../data',
                                                                       format='csv',
                                                                       train='sent140.train.mini.csv',
                                                                       validation='sent140.dev.csv',
                                                                       test='sent140.test.csv',
                                                                       fields=csv_fields,
                                                                       skip_header=False)


txt_field.build_vocab(train_data, dev_data, max_size=100000,
                      vectors='glove.twitter.27B.200d', unk_init = torch.Tensor.normal_)
label_field.build_vocab(train_data)

train_iter, dev_iter, test_iter = torchtext.data.BucketIterator.splits(datasets=(train_data, dev_data, test_data),
                                            batch_sizes=(50,50,50),  # batch sizes of train, dev, test
                                            sort_key=lambda x: len(x.TweetText), # how to sort text
                                            device=device,
                                            sort_within_batch=True,
                                            repeat=False)

In [155]:
# --- Model, Loss, Optimizer Initialization ---

PAD_IDX = txt_field.vocab.stoi[txt_field.pad_token]
UNK_IDX = txt_field.vocab.stoi[txt_field.unk_token]
# WRITE CODE HERE
hidden_dim, batch_size = 30, 50
model = RNN(hidden_dim, batch_size)
print(model)

# Copy the pretrained embeddings into the model
pretrained_embeddings = txt_field.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

# Fix the <UNK> and <PAD> tokens in the embedding layer
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


model = model.to(device)
criterion = criterion.to(device)

RNN(
  (embedding): Embedding(30116, 200)
  (lstm): LSTM(200, 30, num_layers=5)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)


In [156]:
# --- Train Loop ---
for epoch in range(N_EPOCHS):
    start_time = time.time()
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in train_iter:
        # WRITE CODE HERE
        sequence_len=torch.max(batch.TweetText[1])
        
        model.zero_grad()
        outputs = model(batch.TweetText[0], sequence_len)
        
        epoch_acc += get_accuracy(outputs, batch.Label)
        loss = criterion(outputs, batch.Label)
        epoch_loss+=loss
        loss.backward()
        optimizer.step()
        

    train_loss, train_acc = (epoch_loss / len(train_iter), epoch_acc / len(train_iter))
    valid_loss, valid_acc = evaluate(model, dev_iter, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 32s
	Train Loss: 0.594 | Train Acc: 68.28%
	 Val. Loss: 0.514 |  Val. Acc: 76.01%
Epoch: 02 | Epoch Time: 0m 30s
	Train Loss: 0.347 | Train Acc: 85.75%
	 Val. Loss: 0.552 |  Val. Acc: 74.45%
Epoch: 03 | Epoch Time: 0m 29s
	Train Loss: 0.144 | Train Acc: 94.84%
	 Val. Loss: 0.836 |  Val. Acc: 72.12%
Epoch: 04 | Epoch Time: 0m 29s
	Train Loss: 0.064 | Train Acc: 97.92%
	 Val. Loss: 1.109 |  Val. Acc: 70.11%
Epoch: 05 | Epoch Time: 0m 29s
	Train Loss: 0.034 | Train Acc: 99.00%
	 Val. Loss: 1.388 |  Val. Acc: 69.67%
